In [2]:
# WANT: Proj SOG

# Model:

# - SOG/GP TY
# - SOG/GP L3-L5
# - Opp Team Allowed SOG/GP TY
# - Opp Team Allowed SOG/GP L3-L5

# Additional:

# - Any Starting Opp D out
# - Starting Opp G out
# - Back to back games
# - Home vs Away
# - Penalties

homeTeam = "EDM"
awayTeam = "BOS"


In [3]:
# GET TEAM DATA FOR ROSTER PROJ SOG

import pandas as pd

teamUrl = "https://www.hockey-reference.com/teams/" + homeTeam + "/"
fullPageData = pd.read_html(teamUrl)
homeTeamData = fullPageData[1]
homeRosterData = fullPageData[4]
homeRosterData.columns = homeRosterData.columns.droplevel()

# Average shots per game
homeAvgS = homeTeamData['S'].iloc[0] / homeTeamData['GP'].iloc[0]

homeSA = homeTeamData['SA'].iloc[0] / homeTeamData['GP'].iloc[0]
print(f"Opp SA avg: {homeSA}")

SGPData = homeRosterData.loc[:,["Player", "GP","S"]]
SGPData['S/GP'] = SGPData['S']/SGPData['GP']
SGPData.drop(SGPData.tail(1).index,inplace=True)
SGPData['S/GP'] = pd.to_numeric(SGPData['S/GP'])
SGPData = SGPData.loc[SGPData['S/GP'] >= 1.5]
SGPData['H/A'] = 'H'
SGPData

Opp SA avg: 31.71014492753623


,Player,GP,S,S/GP,H/A
0,Connor McDavid,69,299,4.333333,H
1,Leon Draisaitl,67,206,3.074627,H
2,Ryan Nugent-Hopkins,69,166,2.405797,H
3,Zach Hyman,66,239,3.621212,H
4,Tyson Barrie,61,115,1.885246,H
5,Darnell Nurse,69,167,2.420290,H
6,Evan Bouchard,69,136,1.971014,H
7,Warren Foegele,54,102,1.888889,H
9,Evander Kane,28,96,3.428571,H
14,Kailer Yamamoto,45,72,1.600000,H


In [4]:
# GET OPPONENT DATA
# INCLUDES:
#  -  OPPONENTS SHOTS ALLOWED AVERAGE
#  -  LEAGUE SHOTS ALLOWED AVERAGE

awayUrl = "https://www.hockey-reference.com/teams/" + awayTeam + "/"
awayFullPageData = pd.read_html(awayUrl)
awayTeamData = awayFullPageData[1]
awayRosterData = awayFullPageData[4]
awayRosterData.columns = awayRosterData.columns.droplevel()

# Average shots per game
awayAvgS = awayTeamData['S'].iloc[0] / awayTeamData['GP'].iloc[0]

awaySA = awayTeamData['SA'].iloc[0] / awayTeamData['GP'].iloc[0]
leagueAvgSA = awayTeamData['SA'].iloc[1] / awayTeamData['GP'].iloc[1]
print(f"Opp SA avg: {awaySA}")
print(f"League SA avg: {leagueAvgSA}")

awayData = awayRosterData.loc[:,["Player", "GP","S"]]
awayData['S/GP'] = awayData['S']/awayData['GP']
awayData.drop(awayData.tail(1).index,inplace=True)
awayData['S/GP'] = pd.to_numeric(awayData['S/GP'])
awayData = awayData.loc[awayData['S/GP'] >= 1.5]
awayData['H/A'] = 'A'
SGPData = pd.concat([SGPData, awayData], axis=0)
SGPData

Opp SA avg: 29.597014925373134
League SA avg: 31.455882352941178


,Player,GP,S,S/GP,H/A
0,Connor McDavid,69,299,4.333333,H
1,Leon Draisaitl,67,206,3.074627,H
2,Ryan Nugent-Hopkins,69,166,2.405797,H
3,Zach Hyman,66,239,3.621212,H
4,Tyson Barrie,61,115,1.885246,H
5,Darnell Nurse,69,167,2.420290,H
6,Evan Bouchard,69,136,1.971014,H
7,Warren Foegele,54,102,1.888889,H
9,Evander Kane,28,96,3.428571,H
14,Kailer Yamamoto,45,72,1.600000,H


In [5]:
# GET AVG SOG OF LAST 5 GAME
# GET PERCENT OF SOG BASED ON TEAM TOTAL AVG SOG
# GET PERCENT OF SOG BASED ON TEAM TOTAL AVG SOG AND LAST 5 GAME AVG SOG

import numpy as np

playerL5AVG = []
playerTeamSPerc = []
playerTeamSPercL5 = []
for index, row in SGPData.iterrows():
    player = row['Player']
    sGP = row['S/GP']
    homeAway = row['H/A']
    if (homeAway == 'H'):
        playerTeamSPerc.append((sGP/homeAvgS)*100)
    else:
        playerTeamSPerc.append((sGP/awayAvgS)*100)
    print(player)
    url = 'https://www.hockey-reference.com/players/'
    nameSplit = player.split()
    url += nameSplit[1][0]
    url += '/'
    url += nameSplit[1][:5]
    url += nameSplit[0][:2]
    url += '01.html'
    url = url.lower()
    print(url)
    try:
        playerL5 = pd.read_html(url, encoding="iso-8859-1")
        playerL5 = playerL5[0]
        playerL5.columns = playerL5.columns.droplevel()
        playerL5.drop(playerL5.tail(1).index,inplace=True)
        playerL5 = pd.to_numeric(playerL5.loc[:,"S"])
        if (playerL5.mean() > 10):
            playerL5AVG.append(np.nan)
            playerTeamSPercL5.append(np.nan)
        else:
            playerL5AVG.append(playerL5.mean())
            if (homeAway == 'H'):
                playerTeamSPercL5.append((playerL5.mean()/homeAvgS)*100)
            else:
                playerTeamSPercL5.append((playerL5.mean()/awayAvgS)*100)
    except:
        playerL5AVG.append(np.nan)
        playerTeamSPercL5.append(np.nan)



SGPData["S/GP L5"] = playerL5AVG
SGPData["S/GP Team %"] = playerTeamSPerc
SGPData["S/GP L5 Team %"] = playerTeamSPercL5
SGPData

Connor McDavid
https://www.hockey-reference.com/players/m/mcdavco01.html
Leon Draisaitl
https://www.hockey-reference.com/players/d/draisle01.html
Ryan Nugent-Hopkins
https://www.hockey-reference.com/players/n/nugenry01.html
Zach Hyman
https://www.hockey-reference.com/players/h/hymanza01.html
Tyson Barrie
https://www.hockey-reference.com/players/b/barrity01.html
Darnell Nurse
https://www.hockey-reference.com/players/n/nurseda01.html
Evan Bouchard
https://www.hockey-reference.com/players/b/bouchev01.html
Warren Foegele
https://www.hockey-reference.com/players/f/foegewa01.html
Evander Kane
https://www.hockey-reference.com/players/k/kaneev01.html
Kailer Yamamoto
https://www.hockey-reference.com/players/y/yamamka01.html
Jesse Puljujarvi
https://www.hockey-reference.com/players/p/puljuje01.html
Mattias Ekholm
https://www.hockey-reference.com/players/e/ekholma01.html
Nick Bjugstad
https://www.hockey-reference.com/players/b/bjugsni01.html
David Pastrnak
https://www.hockey-reference.com/players

,Player,GP,S,S/GP,H/A,S/GP L5,S/GP Team %,S/GP L5 Team %
0,Connor McDavid,69,299,4.333333,H,2.8,13.051069,8.432999
1,Leon Draisaitl,67,206,3.074627,H,3.4,9.260116,10.240070
2,Ryan Nugent-Hopkins,69,166,2.405797,H,2.6,7.245744,7.830642
3,Zach Hyman,66,239,3.621212,H,2.0,10.906313,6.023570
4,Tyson Barrie,61,115,1.885246,H,1.4,5.677956,4.216499
5,Darnell Nurse,69,167,2.420290,H,2.2,7.289393,6.625928
6,Evan Bouchard,69,136,1.971014,H,1.8,5.936272,5.421213
7,Warren Foegele,54,102,1.888889,H,2.0,5.688928,6.023570
9,Evander Kane,28,96,3.428571,H,2.2,10.326121,6.625928
14,Kailer Yamamoto,45,72,1.600000,H,1.6,4.818856,4.818856


TEMPLATE MATH:

Opponents Allowed SOG Avg

Example : 35.23

Avg SOG

Example: 35.4

Toffoli Avg:

~ 8.7% of the SOG for team

8.7% of 35.23 = Proj 3.065 SOG

Toffoli AVG L5:

~ 9.6%

9.6% of 35.23 = 3.382 SOG

AVG OF ALL SOG AVGS:

3.382 + 3.065 + 3.070 + 3.4 = Aggregate project SOG 3.229

In [6]:
# GETS PROJECTED SOG BASED ON AVG PERCENT OF SOG FROM TEAM TOTAL AND ALLOWED SOG FROM OPPONENT
# GETS SAME AS ABOVE, JUST L5 GAMES

projSOG = []
projSOGL5 = []
for index, row in SGPData.iterrows():
    sPerc = row['S/GP Team %']/100
    sPercL5 = row['S/GP L5 Team %']/100
    projSOG.append(sPerc*awaySA)
    projSOGL5.append(sPercL5*awaySA)
    
SGPData["Proj S"] = projSOG
SGPData["Proj S L5"] = projSOGL5
SGPData

,Player,GP,S,S/GP,H/A,S/GP L5,S/GP Team %,S/GP L5 Team %,Proj S,Proj S L5
0,Connor McDavid,69,299,4.333333,H,2.8,13.051069,8.432999,3.862727,2.495916
1,Leon Draisaitl,67,206,3.074627,H,3.4,9.260116,10.240070,2.740718,3.030755
2,Ryan Nugent-Hopkins,69,166,2.405797,H,2.6,7.245744,7.830642,2.144524,2.317636
3,Zach Hyman,66,239,3.621212,H,2.0,10.906313,6.023570,3.227943,1.782797
4,Tyson Barrie,61,115,1.885246,H,1.4,5.677956,4.216499,1.680505,1.247958
5,Darnell Nurse,69,167,2.420290,H,2.2,7.289393,6.625928,2.157443,1.961077
6,Evan Bouchard,69,136,1.971014,H,1.8,5.936272,5.421213,1.756959,1.604517
7,Warren Foegele,54,102,1.888889,H,2.0,5.688928,6.023570,1.683753,1.782797
9,Evander Kane,28,96,3.428571,H,2.2,10.326121,6.625928,3.056224,1.961077
14,Kailer Yamamoto,45,72,1.600000,H,1.6,4.818856,4.818856,1.426238,1.426238


In [7]:
# GETS AGGREGATE PROJ SOG BASED ON THE STATS BELOW
#  -  AVG SOG
#  -  AVG SOG L5 GAMES
#  -  PROJ SOG BASED ON OPPONENT AND WHOLE SEASON
#  -  PROJ SOG BASED ON OPPONENT AND AVG L5 GAMES

aggProjSOG = []
for index, row in SGPData.iterrows():
    aggProj = row['S/GP'] + row['S/GP L5'] + row['Proj S'] + row['Proj S L5']
    aggProjSOG.append(aggProj/4)
    
SGPData["Agg Proj S"] = aggProjSOG
SGPData

,Player,GP,S,S/GP,H/A,S/GP L5,S/GP Team %,S/GP L5 Team %,Proj S,Proj S L5,Agg Proj S
0,Connor McDavid,69,299,4.333333,H,2.8,13.051069,8.432999,3.862727,2.495916,3.372994
1,Leon Draisaitl,67,206,3.074627,H,3.4,9.260116,10.240070,2.740718,3.030755,3.061525
2,Ryan Nugent-Hopkins,69,166,2.405797,H,2.6,7.245744,7.830642,2.144524,2.317636,2.366989
3,Zach Hyman,66,239,3.621212,H,2.0,10.906313,6.023570,3.227943,1.782797,2.657988
4,Tyson Barrie,61,115,1.885246,H,1.4,5.677956,4.216499,1.680505,1.247958,1.553427
5,Darnell Nurse,69,167,2.420290,H,2.2,7.289393,6.625928,2.157443,1.961077,2.184702
6,Evan Bouchard,69,136,1.971014,H,1.8,5.936272,5.421213,1.756959,1.604517,1.783123
7,Warren Foegele,54,102,1.888889,H,2.0,5.688928,6.023570,1.683753,1.782797,1.838860
9,Evander Kane,28,96,3.428571,H,2.2,10.326121,6.625928,3.056224,1.961077,2.661468
14,Kailer Yamamoto,45,72,1.600000,H,1.6,4.818856,4.818856,1.426238,1.426238,1.513119
